In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from pathlib import Path
from tqdm import tqdm
import pdfplumber

In [22]:
OEP = "https://web.oep.org.bo"
LISTAS = Path("listas/2025")

In [45]:
def descargar_listas():
    os.makedirs(LISTAS, exist_ok=True)
    # html = BeautifulSoup(
    #     requests.get(f"{OEP}/elecciones-generales-2025/nacional-2025/").text,
    #     "html.parser",
    # )
    # for entry in html.select(".et_pb_module"):
    #     header = entry.select("h5")
    #     if (
    #         header
    #         and "listado de candidaturas habilitadas" in header[0].get_text().lower()
    #     ):
    #         table = entry.select("tbody")[0]
    #         break
    # documentos = [
    #     {
    #         "partido": row.select("td")[0].get_text(),
    #         "url": f"{OEP}/{row.select('td a')[0]['href']}",
    #     }
    #     for row in table.select("tr")
    # ]
    documentos = [
        {
            "partido": re.search(
                r"-de-([A-Z-]+)-\d{2}-\d{2}-\d{4}", f["source_url"]
            ).group(1),
            "url": f["source_url"],
        }
        for f in requests.get(
            f"{OEP}/wp-json/wp/v2/media?media_type=application&per_page=100&search=16-08-2025"
        ).json()
        if "reporte-de-candidatos-habilitados" in f["slug"]
    ]
    for documento in tqdm(documentos):
        # partido_short = re.search(r"\((.*?)\)", documento["partido"]).group(1).lower()
        path = LISTAS / f"{documento['partido']}.pdf"
        with open(path, "wb") as f:
            response = requests.get(documento["url"])
            f.write(response.content)
        documento["path"] = path
    return documentos

In [46]:
listas = descargar_listas()

100%|██████████| 10/10 [00:18<00:00,  1.88s/it]


In [47]:
listas

[{'partido': 'AP',
  'url': 'https://web.oep.org.bo/wp-content/uploads/2025/08/Reporte-de-Candidatos-Habilitados-de-AP-16-08-2025-150641.pdf',
  'path': PosixPath('listas/2025/AP.pdf')},
 {'partido': 'APB-SUMATE',
  'url': 'https://web.oep.org.bo/wp-content/uploads/2025/08/Reporte-de-Candidatos-Habilitados-de-APB-SUMATE-16-08-2025-150811.pdf',
  'path': PosixPath('listas/2025/APB-SUMATE.pdf')},
 {'partido': 'BIA-YUQUI',
  'url': 'https://web.oep.org.bo/wp-content/uploads/2025/08/Reporte-de-Candidatos-Habilitados-de-BIA-YUQUI-16-08-2025-150839.pdf',
  'path': PosixPath('listas/2025/BIA-YUQUI.pdf')},
 {'partido': 'FP',
  'url': 'https://web.oep.org.bo/wp-content/uploads/2025/08/Reporte-de-Candidatos-Habilitados-de-FP-16-08-2025-150916.pdf',
  'path': PosixPath('listas/2025/FP.pdf')},
 {'partido': 'LIBRE',
  'url': 'https://web.oep.org.bo/wp-content/uploads/2025/08/Reporte-de-Candidatos-Habilitados-de-LIBRE-16-08-2025-150956.pdf',
  'path': PosixPath('listas/2025/LIBRE.pdf')},
 {'partido'

In [49]:
def collect_values(pdf_path):

    pdf = pdfplumber.open(pdf_path)
    candidaturas, departamentos, tables = [], [], []
    
    for page in pdf.pages:
        text = page.extract_text()
        candidaturas.extend(re.findall(r"^Candidata\(o\) a:\s*(.*)", text, re.MULTILINE))
        departamentos.extend(re.findall(r"^Departamento:\s*(.*)", text, re.MULTILINE))
        page_tables = page.extract_tables()
        for table in page_tables:
            # si es una tabla de candidatos
            if len(table[0]) == 10:
                # Si es una nueva tabla
                if table[0][0] == "Orden":
                    # Si incluye valores
                    if len(table) > 1:
                        tables.append(table[1:])
                    # Si sólo es el cabezal
                    else:
                        tables.append([])
                # Si es la continuación de una tabla ya ingresada
                else:
                    tables[-1].extend(table)
    return candidaturas, departamentos, tables

In [50]:
def extraer_datos(pdf_path, partido):
    habilitados = []

    candidaturas, departamentos, tables = collect_values(pdf_path)
    columns = [
        "candidatura",
        "departamento",
        "orden",
        "posición",
        "titularidad",
        "nombre_completo",
        "nro_documento",
        "genero",
        "edad",
        "fecha_nacimiento",
        "descripcion",
        "observacion",
    ]
    for candidatura, departamento, table in zip(
        candidaturas, 2 * ["Nacional"] + departamentos, tables
    ):
        for row in table:
            entry = {
                key: value
                for key, value in zip(columns, [candidatura, departamento] + row)
            }
            habilitados.append(entry)

    df = pd.DataFrame(habilitados)
    df["fecha_nacimiento"] = pd.to_datetime(df.fecha_nacimiento, format="%d/%m/%Y")
    df = df.astype(
        {
            "candidatura": "category",
            "departamento": "category",
            "orden": int,
            "posición": int,
            "titularidad": "category",
            "nombre_completo": str,
            "nro_documento": str,
            "genero": "category",
            "edad": int,
        }
    )
    df.insert(0, "partido", partido)
    return df

In [51]:
df = pd.concat([extraer_datos(lista["path"], lista["partido"]) for lista in listas])

In [52]:
df

,partido,candidatura,departamento,orden,posición,titularidad,nombre_completo,nro_documento,genero,edad,fecha_nacimiento,descripcion,observacion
0,AP,Presidente,Nacional,1,1,TITULAR,ANDRONICO RODRIGUEZ LEDEZMA,7971986,M,36,1988-11-11,,
1,AP,Vicepresidente,Nacional,1,1,TITULAR,MARIANA PRADO NOYA,4806169,F,43,1982-04-20,,
2,AP,Senadores,Chuquisaca,1,1,TITULAR,JIMENA VILLALTA QUINTEROS,5649993,F,42,1983-02-02,,
3,AP,Senadores,Chuquisaca,2,1,SUPLENTE,ALEX BLACUTT PANDAL,1145255,M,52,1973-01-01,,
4,AP,Senadores,Chuquisaca,3,2,TITULAR,DIEGO IGNACIO CUELLAR SERRUDO,10349763,M,30,1994-11-03,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,UNIDAD,Diputados de Circunscripciones Uninominales,Pando,2,1,TITULAR,ABIGAIL IDAGUA MAZA,7625228,F,44,1980-12-29,Circunscripción 63,
320,UNIDAD,Diputados de Circunscripciones Uninominales,Pando,3,1,SUPLENTE,RONALD ARCE DURI,4173245,M,48,1976-11-24,Circunscripción 63,
321,UNIDAD,Diputados de Circunscripciones Especiales,Pando,1,1,TITULAR,VIRGINIA MIRANDA COELHO,4200980,F,36,1989-04-20,,
322,UNIDAD,Diputados de Circunscripciones Especiales,Pando,2,1,SUPLENTE,JESUS CHIPANA DARA,7596142,M,53,1971-12-24,,


In [53]:
prev = pd.read_parquet("datos/2025.parquet")

In [59]:
df["nuevo"] = ~df.nro_documento.isin(prev.nro_documento)

In [60]:
df

,partido,candidatura,departamento,orden,posición,titularidad,nombre_completo,nro_documento,genero,edad,fecha_nacimiento,descripcion,observacion,nuevo
0,AP,Presidente,Nacional,1,1,TITULAR,ANDRONICO RODRIGUEZ LEDEZMA,7971986,M,36,1988-11-11,,,False
1,AP,Vicepresidente,Nacional,1,1,TITULAR,MARIANA PRADO NOYA,4806169,F,43,1982-04-20,,,False
2,AP,Senadores,Chuquisaca,1,1,TITULAR,JIMENA VILLALTA QUINTEROS,5649993,F,42,1983-02-02,,,False
3,AP,Senadores,Chuquisaca,2,1,SUPLENTE,ALEX BLACUTT PANDAL,1145255,M,52,1973-01-01,,,False
4,AP,Senadores,Chuquisaca,3,2,TITULAR,DIEGO IGNACIO CUELLAR SERRUDO,10349763,M,30,1994-11-03,,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,UNIDAD,Diputados de Circunscripciones Uninominales,Pando,2,1,TITULAR,ABIGAIL IDAGUA MAZA,7625228,F,44,1980-12-29,Circunscripción 63,,False
320,UNIDAD,Diputados de Circunscripciones Uninominales,Pando,3,1,SUPLENTE,RONALD ARCE DURI,4173245,M,48,1976-11-24,Circunscripción 63,,False
321,UNIDAD,Diputados de Circunscripciones Especiales,Pando,1,1,TITULAR,VIRGINIA MIRANDA COELHO,4200980,F,36,1989-04-20,,,True
322,UNIDAD,Diputados de Circunscripciones Especiales,Pando,2,1,SUPLENTE,JESUS CHIPANA DARA,7596142,M,53,1971-12-24,,,False


In [62]:
df.to_parquet("datos/2025.parquet")